<a href="https://colab.research.google.com/github/GonzalezMada/machine-learning-zoomcamp2025-homework/blob/main/home_work_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import zipfile
import statistics

# --- 1. CONFIGURATION & REPRODUCIBILITY ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. DATA DOWNLOAD & PREPARATION ---
if not os.path.exists('data.zip'):
    print("Downloading data...")
    !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

if not os.path.exists('data'):
    print("Unzipping data...")
    with zipfile.ZipFile('data.zip', 'r') as zip_ref:
        zip_ref.extractall('.')

# --- 3. MODEL DEFINITION ---
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        # Conv Layer: 3 input channels, 32 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)

        # Calculation for linear layer input:
        # Image: 200x200 -> Conv(3x3) -> 198x198 -> Pool(2x2) -> 99x99
        # Flatten size: 32 channels * 99 * 99 = 313,632
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1) # Flatten all dims except batch
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = HairClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Check Params (Q2 Verification)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

# --- 4. PHASE 1 TRAINING (Questions 3 & 4) ---
# Data Transforms
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Test transforms are usually the same as Train without augmentation
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('./data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('./data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
validation_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=2)

print("\n--- Starting Phase 1 Training ---")
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

# --- ANSWERS Q3 & Q4 ---
median_train_acc = statistics.median(history['acc'])
std_train_loss = statistics.stdev(history['loss'])

print("\n" + "="*30)
print(f"Q3 Median Training Accuracy: {median_train_acc:.4f}")
print(f"Q4 Std Dev Training Loss:    {std_train_loss:.4f}")
print("="*30 + "\n")


# --- 5. PHASE 2 TRAINING WITH AUGMENTATION (Questions 5 & 6) ---
print("--- Starting Phase 2 (Augmentation) ---")

# Define Augmented Transforms
aug_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)), # Resize is redundant if RandomResizedCrop is used correctly, but good for safety
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Reload Train Dataset with Augmentation
train_dataset_aug = datasets.ImageFolder('./data/train', transform=aug_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=2)

# Continue training for 10 more epochs (Total 20)
# Note: We do NOT re-initialize the model or optimizer.
aug_history = {'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs): # Another 10 epochs
    model.train()
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation (No changes to validation logic)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    aug_history['val_loss'].append(val_epoch_loss)
    aug_history['val_acc'].append(val_epoch_acc)

    print(f"Augmented Epoch {epoch+1}/{num_epochs} | Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f}")

# --- ANSWERS Q5 & Q6 ---
mean_aug_test_loss = statistics.mean(aug_history['val_loss'])
mean_aug_test_acc_last5 = statistics.mean(aug_history['val_acc'][5:])

print("\n" + "="*30)
print(f"Q5 Mean Test Loss (Augmented): {mean_aug_test_loss:.4f}")
print(f"Q6 Avg Test Acc (Last 5 epochs): {mean_aug_test_acc_last5:.4f}")
print("="*30)

Using device: cuda
--2025-12-01 18:06:23--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T18%3A44%3A08Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T17%3A44%3A05Z&ske=2025-12-01T18%3A44%3A08Z&sks=b&skv=2018-11-09&sig=DeTKET%2BbBNqZ9fmqfE9Vqpw%2BSv5vawbfERaHcVGRpZ4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYxNDE4MywibmJmIjoxNzY0NjEyMzgzLCJwYXRoIjoicmVsZWFzZWF